In [1]:
from dotenv import load_dotenv

import os

load_dotenv()

True

## Checando Tools 

Vamos importar as tools criadas anteriormente! 

In [2]:
from tools import *

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
trends = create_trends_tool()
trends_text = trends.invoke({"country": "brazil", 
                             "head": 3})
trends_list = to_list(trends_text)
trends_list

['Fies', 'Sisu 2025', 'Real Sociedad']

In [5]:
contents_news = get_serper_with_scrapping(trends_list, k=5)

100%|██████████| 3/3 [00:09<00:00,  3.17s/it]


In [6]:
contents_news

{'Fies': [{'title': 'Prouni e Fies: Ministro Camilo Santana divulga datas do primeiro semestre',
   'link': 'https://g1.globo.com/educacao/noticia/2025/01/13/datas-de-prouni-e-fies-do-primeiro-semestre.ghtml',
   'section': 'MEC divulga calendário do ProUni e do Fies',
   'date': 'há 10 horas',
   'source': 'g1 - O portal de notícias da Globo',
   'position': 1,
   'content': " Prouni e Fies: Ministro Camilo Santana divulga datas do primeiro semestre | Educação | G1\n                   Educação                                       Prouni e Fies: Ministro Camilo Santana divulga datas do primeiro semestre     Cronogramas ainda não são oficiais, já que não há editais publicados para os programas. Datas foram informadas em evento sobre o Enem 2024 que acontece em Brasília.              Por Redação g1    13/01/2025 10h39    Atualizado  13/01/2025                                                  1 de 2\rCronograma PROUNI 2025 — Foto: Reprodução/Ministério da Educação     Cronograma PROUNI 2

## Construção do Agent

### Estado do Grafo

In [7]:
from typing_extensions import TypedDict
from typing import List, Dict

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        country: country of the trends
        head: number of trends to get
        k: number of news to get
        trends: list of trends
        news: dict of news about each trend
        newsletter: newsletter from the news
    """
    country: str
    head: int
    k: int
    trends: List[str]
    news: Dict
    newsletter: str

### Chains

In [8]:
def create_newletter():
    pass

### Nodes

In [ ]:
def get_trends(state):
    """
    Get trends from Google Trends

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): 
    """
    print("---GET TRENDS---")
    country = state["country"]
    head = state["head"]

    # Get trends
    trends = create_trends_tool()
    trends_text = trends.invoke({"country": country, 
                                 "head": head})
    trends_list = to_list(trends_text)
    return {"country": country, 
            "head": head, 
            "trends": trends_list}

def get_serper_and_scrapping(state):
    """
    Get news from SerpAPI and scrap the content

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): 
    """
    print("---GET SERPER AND SCRAPPING---")
    trends = state["trends"]
    k = state["k"]
    
    # Get news
    contents_news = get_serper_with_scrapping(queries=trends, 
                                              k=k)
    return {"news": contents_news}

def create_newletter(state):
    """
    Create a newsletter from the news

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): 
    """

    print("---CREATE NEWSLETTER---")
    news = state["news"]
    newsletter = create_newsletter(news)
    
    return state
